# Experiment on replicating `Modeling rain-driven overland flow`
The goal is to simulate rain accumulating on a surface and flowing, using the Poiseuille friction model which the [paper](https://arxiv.org/pdf/1609.04711) found most suitable for these conditions.

1. Geometry = a tilted rectangle, not a basin

In [1]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import *
from ngsolve.webgui import Draw
import numpy as np


L  = 4.04 # flume length  [m]
W  = 0.115 # flume width   [m]
SlopeDeg = 5 # (%)  ➔  2 %  ⇒ 0.02
S0   = SlopeDeg/100 # tan α  (positive downslope)

# create a flat rectangle; the slope only appears in the source term
geo = Rectangle(L,W).Face()
geo.edges.Min(X).name="inlet"
geo.edges.Max(X).name="outlet"
geo.edges.Min(Y).name = "wall"
geo.edges.Max(Y).name = "wall"
geo = OCCGeometry(geo, dim=2)
mesh = Mesh(geo.GenerateMesh(maxh=0.05))
# mesh.Curve(3)
Draw(mesh)


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

2. Unknowns, rainfall source and friction law

In [2]:
order = 2
fes   = L2(mesh,order=order)**3 # (h, h u, h v)

U,V = fes.TnT() # "Trial" and "Test" function
h, hu, hv = U

g  = 9.81 # [m/s²]
nu = 1e-6 # kinematic viscosity of water  [m²/s]
I_mm_h = 250 # rainfall intensity [mm h⁻¹]
R = I_mm_h/1000/3600 # convert to m s⁻¹  (≈ 6.94 × 10⁻⁶)
eps = 1e-6


In [3]:
min_h_friction = 1e-6 # Minimum water depth for friction denominator (can be tuned)

def Sf_poiseuille(U_components_cf):
    h_cf, hu_cf, hv_cf = U_components_cf # These are CoefficientFunctions

    # Regularize h to avoid division by zero or very small numbers
    h_regularized = Max(h_cf, min_h_friction)

    # Calculate S_f components, assuming hu_cf is flux qx = h*u, hv_cf is flux qy = h*v
    # Sf = (3*nu/g) * (q / h_reg^3)
    S_f_x = (3 * nu / g) * (hu_cf / h_regularized**3)
    S_f_y = (3 * nu / g) * (hv_cf / h_regularized**3) # If your model is 1D in x, hv_cf might be 0

    return CF((S_f_x, S_f_y))

In [4]:
n = specialcf.normal(mesh.dim)
def Max(a, b):
    """point-wise maximum for CoefficientFunctions"""
    return IfPos(a-b, a, b)

In [5]:
def F(U):
    h, hvx, hvy = U
    vx = hvx/h
    vy = hvy/h
    return CF(((hvx,hvy),
               (hvx*vx + 0.5*g*h**2, hvx*vy),
               (hvx*vy, hvy*vy + 0.5*g*h**2)),dims=(3,2))

In [6]:
n = specialcf.normal(mesh.dim)
def Max(u,v):
    return IfPos(u-v,u,v)
def Fmax(A,B): # max. characteristic speed:
    ha, hua, hva = A
    hb, hub, hvb = B
    vnorma = sqrt(hua**2+hva**2)/ha
    vnormb = sqrt(hub**2+hvb**2)/hb
    return Max(vnorma+sqrt(g*A[0]),vnormb+sqrt(g*B[0]))

def Fhatn(U): # numerical flux
    Uhat = U.Other(bnd=Ubnd)
    return (0.5*F(U)+0.5*F(Uhat))*n + Fmax(U,Uhat)/2*(U-Uhat)

In [7]:

U0 = CF((eps, 0, 0))
Ubnd = CF((h, -hu, -hv))

In [28]:
print(F(U).shape)
print(F(U).trans.shape)
print(Grad(V).shape)
print(Grad(V).trans.shape)


(3, 2)
(2, 3)
(3, 2)
(2, 3)


3. DG bilinear plus source terms (rain, bed-slope, friction)

In [8]:
def DGBilinearForm(fes,F,Fhatnd, Ubnd):
    a = BilinearForm(fes, nonassemble=True)
    a += - InnerProduct(F(U),Grad(V)) * dx
    a += InnerProduct(Fhatn(U),V) * dx(element_boundary=True)
    return a

a = DGBilinearForm(fes,F,Fhatn, Ubnd)

In [9]:
from DGdiffusion import AddArtificialDiffusion

artvisc = Parameter(1.0)
if order > 0:
    AddArtificialDiffusion(a,Ubnd,artvisc,compile=True)

In [10]:
invMA = fes.Mass(1).Inverse() @ a.mat


In [11]:
gfu = GridFunction(fes)
gfu.Set(U0)

gfh, gfhu, gfhv = gfu

min_depth = 1e-5
gfvu = IfPos(gfh - min_depth, gfhu/gfh, 0)
gfvv = IfPos(gfh - min_depth, gfhv/gfh, 0)

momentum = CF((gfhu, gfhv))
velocity = CF((gfvu, gfvv))

scenes = [ 
    Draw(momentum, mesh, "mom"),
    Draw(velocity, mesh, "vel"),
    Draw(gfh, mesh, "h") 
]

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [15]:
def Source(gfu_arg):
    h_comp  = gfu_arg.components[0]
    hu_comp = gfu_arg.components[1]
    hv_comp = gfu_arg.components[2] # Ensure this exists if used

    # Rainfall contribution
    srain_h  = R      # Rainfall intensity (m/s)
    srain_hu = 0.0
    srain_hv = 0.0    # Assuming no momentum from rain in this simplified model

    # Bed-slope contribution: g*h*S0
    # Use a regularized h_comp for slope if h_comp can become problematic (e.g., negative)
    h_for_slope = Max(h_comp, eps) # eps is your small positive number for initial depth
    sslope_hu = g * h_for_slope * S0
    sslope_hv = 0.0   # Assuming slope is only in x-direction

    # Friction contribution: -g*h*S_f
    # Sf_poiseuille returns (S_f_x, S_f_y)
    friction_S_f_tuple = Sf_poiseuille((h_comp, hu_comp, hv_comp))

    # Use a regularized h_comp for the -g*h*S_f term as well
    h_for_friction_factor = Max(h_comp, eps)
    s_friction_hu = -g * h_for_friction_factor * friction_S_f_tuple[0]
    s_friction_hv = -g * h_for_friction_factor * friction_S_f_tuple[1]

    # Total source terms
    total_source_h  = srain_h
    total_source_hu = srain_hu + sslope_hu + s_friction_hu
    total_source_hv = srain_hv + sslope_hv + s_friction_hv # If 2D

    return CF((0.0, 0.0, 0.0)) #return CF((total_source_h, total_source_hu, total_source_hv))

4. Time integrator

In [16]:
def TimeLoop(a, gfu, dt, T, Source, nsamplings=100, scenes=scenes,
             multidim_draw=False, md_nsamplings=20):
    if multidim_draw:
        gfu_t = GridFunction(gfu.space, multidim=True)
    res = gfu.vec.CreateVector()
    fes = a.space
    t = 0; i = 0
    nsteps = int(ceil(T / dt))
    invma = fes.Mass(1).Inverse() @ a.mat

    if multidim_draw:
        gfu_t.AddMultiDimComponent(gfu.vec)
    mass_inv = fes.Mass(1).Inverse()
    log_interval = 50
    max_h_threshold = 1.0
    with TaskManager():
        while t <= T - 0.5 * dt:
            res.data = invma * gfu.vec
            lf_source = LinearForm(fes)
            lf_source += InnerProduct(Source(gfu), fes.TestFunction()) * dx
            lf_source.Assemble()
            source_term = mass_inv * lf_source.vec
            res.data -= source_term
            gfu.vec.data -= dt * res
            t += dt
            if (i + 1) % max(int(nsteps / nsamplings), 1) == 0:
                for s in scenes:
                    s.Redraw()
            if (i + 1) % log_interval == 0:
                h_coeffs = gfu.components[0].vec.FV().NumPy()
                hu_coeffs = gfu.components[1].vec.FV().NumPy()
                hv_coeffs = gfu.components[2].vec.FV().NumPy() 
                print(f"\nStep: {i+1}, Time: {t:.6f}")
                print(f"  h min: {np.min(h_coeffs):.3e}, h max: {np.max(h_coeffs):.3e}")
                print(f"  hu min: {np.min(hu_coeffs):.3e}, hu max: {np.max(hu_coeffs):.3e}")
                # print(f"  hv min: {np.min(hv_coeffs):.3e}, hv max: {np.max(hv_coeffs):.3e}") # Uncomment if hv is used
                   
            if multidim_draw and (i + 1) % max(int(nsteps / md_nsamplings), 1) == 0:
                gfu_t.AddMultiDimComponent(gfu.vec)
            i += 1
            print("\rt = {:.6f}".format(t), end="")
    for s in scenes:
        s.Redraw()
    if multidim_draw:
        return gfu_t
gfu.Set(U0)
dt = 0.0025
T = 15

In [18]:
Draw(gfu_t.components[0], mesh, "h", interpolate_multidim=True, animate=True,
     deformation=True, settings={"camera": {"transformations": [
         {"type": "rotateX", "angle": -20},
         {"type": "rotateZ", "angle": 0}
     ]}}, autoscale=True)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'camera': {'transformations':…

BaseWebGuiScene